<a href="https://colab.research.google.com/github/Yewon9/STT_JEJU/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers datasets torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import datasets
from datasets import Dataset
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import TrainingArguments, Trainer

In [6]:
data = pd.read_csv('sampled_dataset.csv')  # CSV 파일 경로
dataset = Dataset.from_pandas(data)

In [7]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["Path"])
    batch["speech"] = speech_array
    return batch

In [ ]:
dataset = dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
dataset.save_to_disk("/content/drive/MyDrive/project/intermediate_dataset")
print("중간 결과 저장 완료")

런타임 재실행

In [ ]:
dataset = Dataset.load_from_disk("/content/drive/MyDrive/project/intermediate_dataset")
print("데이터 로드 완료")

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

In [ ]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=16000).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["Text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=["speech", "Text"])

In [ ]:
dataset.save_to_disk("/content/drive/MyDrive/project/processed_dataset")

런타임 재실행

In [ ]:
dataset = Dataset.load_from_disk("/content/drive/MyDrive/project/processed_dataset")

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/project/wav2vec2_finetuning",
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("/content/drive/MyDrive/project/wav2vec2_finetuned_model")
processor.save_pretrained("/content/drive/MyDrive/project/wav2vec2_finetuned_processor")

In [ ]:
eval_results = trainer.evaluate()

# WER (Word Error Rate) 확인
print(f"WER: {eval_results['eval_wer']}")